<a href="https://colab.research.google.com/github/MahzabinC/Plant-Pathology-Detection/blob/main/Plant_Pathology_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os
import torch.nn as  nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torchvision.transforms import Compose, Resize, CenterCrop, RandomCrop,ToTensor, Normalize, Grayscale, RandomRotation,InterpolationMode


import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from PIL import Image



In [2]:
# Converting images into tensors
transform= transforms.Compose([
    transforms.RandomResizedCrop(128),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# Load the CSV file
csv_file = '/content/drive/MyDrive/plant-pathology-2021-fgvc8/train.csv'
data = pd.read_csv(csv_file)

# Get unique labels and create mappings
unique_labels = data['labels'].unique()
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
index_to_label = {idx: label for idx, label in enumerate(unique_labels)}

print("Label to Index Mapping:", label_to_index)
print("Index to Label Mapping:", index_to_label)


Label to Index Mapping: {'healthy': 0, 'scab frog_eye_leaf_spot complex': 1, 'scab': 2, 'complex': 3, 'rust': 4, 'frog_eye_leaf_spot': 5, 'powdery_mildew': 6, 'scab frog_eye_leaf_spot': 7, 'frog_eye_leaf_spot complex': 8, 'rust frog_eye_leaf_spot': 9, 'powdery_mildew complex': 10, 'rust complex': 11}
Index to Label Mapping: {0: 'healthy', 1: 'scab frog_eye_leaf_spot complex', 2: 'scab', 3: 'complex', 4: 'rust', 5: 'frog_eye_leaf_spot', 6: 'powdery_mildew', 7: 'scab frog_eye_leaf_spot', 8: 'frog_eye_leaf_spot complex', 9: 'rust frog_eye_leaf_spot', 10: 'powdery_mildew complex', 11: 'rust complex'}


In [46]:
class PlantDataset(Dataset):
    def __init__(self, csv, image_folder, transform=None,label_mapping=None):
        self.csv = csv
        self.image_folder = image_folder
        self.transform = transform
        self.data = pd.read_csv(self.csv,index_col="image")
        # Create a dictionary mapping labels to numerical indices
        self.label_mapping = label_mapping


    def __len__(self):
        return len(self.data)


    def __getitem__(self, index):
        # Get the image filename from the CSV file
        image_name = self.data.index[index] # Accessing the index instead of values to get image name
        # Construct the full image path
        image_path = os.path.join(self.image_folder, image_name)

        try:
         image = Image.open(image_path).convert("RGB")
         print("meow")
        except FileNotFoundError:
         print(f"Warning: Image file not found: {image_path}")
         # You can choose to skip this image or raise an exception,
         # depending on your desired behavior
         return None, None  # For example, return None if image not found

        #image = Image.open(image_path).convert("RGB")

        label_str = self.data.iloc[index, 0]
        # Convert to numerical index using label_mapping
        label = self.label_mapping[label_str]  # Map label string to numeric index

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, label

In [47]:
# Loading Dataset from drive
complete_dataset=PlantDataset("/content/drive/MyDrive/plant-pathology-2021-fgvc8/train.csv","/content/drive/MyDrive/plant-pathology-2021-fgvc8/train_images",transform,label_to_index)

# Dividing dataset into Train and Test set
train_size = int(0.8 * len(complete_dataset))
test_size = len(complete_dataset) - train_size
train_dataset, test_dataset= random_split(complete_dataset, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
print(train_dataset[2][1])

# Testing if i have images
for batch_idx, (images, labels) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    for i in range(len(images)):
        print(f"  Image {i + 1} - Label: {labels[i].item()}")
        img = images[i].permute(1, 2, 0)
        img_np = img.numpy()
        img_pil = Image.fromarray((img_np * 255).astype('uint8'))
        img_pil.show()

    # You might want to break the loop after printing a few batches
    if batch_idx == 2:  # Print only 3 batches
        break

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)




meow
2
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [33]:
# Defining the model
class ConvolutionalNetwork(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # 1st Convolutional Layer
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 2nd Convolutional Layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 3rd Convolutional Layer
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 4th Convolutional Layer
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layers
        self.fc1 = nn.Linear(128 * 8 * 8, 128)
        self.fc1 = nn.Linear(128, 52)
        self.fc1 = nn.Linear(52, num_classes)


    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = self.maxpool3(self.relu3(self.conv3(x)))
        x = self.maxpool4(self.relu4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc(x))
        x = F.relu(self.fc1(x))
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


num_classes = 12  # we have 12 classes here
model = ConvolutionalNetwork(num_classes)

model

ConvolutionalNetwork(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU()
  (maxpool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=52, out_features=12, bias=True)
)

In [34]:
# Loss Function optimizer
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)


In [37]:
epochs = 10
for i in range(epochs):
    tst_corr = 0

    # Train
    for b, (train_image, train_label) in enumerate(train_loader):
        b += 1  # Start batches at 1
        train_predicted = model(train_image)  # Get predictions from the model
        loss = criterion(train_predicted, train_label)  # Calculate loss


        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print out results every 600 batches
        if b % 600 == 0:
            print(f'Epoch: {i+1}  Batch: {b}  Loss: {loss.item()}')

    # Calculate and print training accuracy
    train_accuracy = trn_corr / len(train_loader.dataset) * 100
    print(f'Epoch {i+1} Training accuracy: {train_accuracy:.2f}%')

    # Test
    with torch.no_grad():  # Disable gradient tracking for testing
        for b, (test_image, test_label) in enumerate(test_loader):

            test_predicted = model(test_image)  # Get predictions
            correct_pred = torch.max(test_predicted.data, 1)[1]  # Get predicted class index
            tst_corr += (correct_pred == test_label).sum().item()  # Accumulate correct predictions

    # Calculate and print test accuracy
    test_accuracy = tst_corr / len(test_loader.dataset) * 100
    print(f'Epoch {i+1} Test accuracy: {test_accuracy:.2f}%')


meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow
meow


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>